## import all the dataset and anonymize all the columns

# read multiple files and anonymize them

In [1]:
from anonymizedf.anonymizedf import anonymize
# import all csv files in a directory
seprator = ";"
import glob
import pandas as pd
# ... (import other necessary modules)

seprator = ";"

for one_filename in glob.glob(r'data1.csv'):
    print(f'loading {one_filename}')
    try:
        new_df = pd.read_csv(one_filename, delimiter=seprator)
        
        if "omschrijving" in new_df.columns:  # Check if "omschrijving" column exists
            an = anonymize(new_df)
            fake_df = (
                an
                .fake_categy("omschrijving", chaining=True)
                # .fake_rakening("rekening_nummer", "tegenrekening_nummer", chaining=True)
                # .fake_categories("rekening_naam", "tegenrekening_naam", chaining=True)
                # .fake_categy("omschrijving")
                # .show_data_frame()
            )
            print(one_filename + "saving")
            fake_df.save(f"ano_{one_filename}")
        else:
            print("Column 'omschrijving' not found in the DataFrame. Skipping 'omschrijving' column...")
            an = anonymize(new_df)  # Continue anonymizing other columns
            fake_df = (
                an
                .fake_rakening("rekening_nummer", "tegenrekening_nummer", chaining=True)
                .fake_categories("rekening_naam", "tegenrekening_naam", chaining=True)
                # .fake_categy("omschrijving")
                # .show_data_frame()
            )
            print(one_filename + "saving")
            fake_df.save(f"ano_{one_filename}")
        
    except Exception as e:
        print("Exception occurred for value '"+ "': "+ str(e))







loading data1.csv
Column 'omschrijving' not found in the DataFrame. Skipping 'omschrijving' column...
{1: 'NL63GKBE5102907469', 2: 'NL61GRVL2701827349', 5: 'NL70QFIQ9734356611', 3: 'NL30UGFI1467561478', 4: 'NL65CCZQ0034189068', 0: 'NL55KNFQ8411987698'}
data1.csvsaving


In [1]:
from faker import Faker
import re
fake=Faker('nl_NL')
postcode_to_fake_postcode= {}
postcode_to_fake_postcode={}
stad_to_fake_stad={}
straat_to_fake_straat ={}

# def _extract_postcode_street_anonymize(text):
#     parts = text.split()

#     # Find the index of the first numeric part
#     numeric_index = None
#     for i, part in enumerate(parts):
#         if part.isnumeric():
#             numeric_index = i
#             break

#     if numeric_index is not None:
#         postcode = parts[numeric_index]
        
#         # Handle postcode longer than four digits
#         if len(postcode) > 4:
#             postcode = postcode[-4:] + parts[numeric_index + 1][:2]

#         fake_postcode = postcode_to_fake_postcode.get(postcode, fake.postcode())
#         postcode_to_fake_postcode[postcode] = fake_postcode
#         # print(postcode)
#         street_name = " ".join(parts[:numeric_index])
#         fake_straatnaam = straat_to_fake_straat.get(street_name, fake.street_name())
#         straat_to_fake_straat[street_name] = fake_straatnaam
#         city_start_index = None
#         for i, part in enumerate(parts):
#             if "%" in part:
#                 city_start_index = i 
#                 break
#         # Extract city from parts
#         city = " ".join(parts[numeric_index + 2:city_start_index])
#         fake_city = stad_to_fake_stad.get(city, fake.city())
#         stad_to_fake_stad[city] = fake_city

#         # Reconstruct the text with anonymized postcode, street, and city
#         reconstructed_text = f"{fake_straatnaam} {fake_postcode} {fake_city} %1"
#         return reconstructed_text
#     else:
#         return text

def _extract_postcode_street_anonymize(text):
    parts = text.split()
    # Find the index of the first numeric part
    numeric_index = None
    for i, part in enumerate(parts):
        if part.isnumeric():
            numeric_index = i
            break

    if numeric_index is not None:
        postcode = parts[numeric_index]
        
        # Handle postcode longer than four digits
        if len(postcode) > 4:
            next_part = parts[numeric_index + 1]
            if next_part.isalpha() and len(next_part) >= 2:
                postcode = postcode[-4:] + next_part[:2]
        #  add regex to extract post codes
        postcode_pattern = r'\d{4}\s[a-zA-Z]{2}\b'
        postcode = re.findall(postcode_pattern, text)
        postcode= ", ".join(postcode)
        fake_postcode = postcode_to_fake_postcode.get(postcode, fake.postcode())
        postcode_to_fake_postcode[postcode] = fake_postcode

        street_name = " ".join(parts[:numeric_index])
        fake_straatnaam = straat_to_fake_straat.get(street_name, fake.street_name())
        straat_to_fake_straat[street_name] = fake_straatnaam

        # Find the index of "%" in parts
        city_start_index = None
        for i, part in enumerate(parts):
            if "%" in part:
                city_start_index = i + 1
                break

        # Extract city from parts
        if city_start_index is not None:
            city = " ".join(parts[-1:city_start_index])  # Exclude the postcode
            fake_city = stad_to_fake_stad.get(city, fake.city())
            stad_to_fake_stad[city] = fake_city
        else:
            city = None
       

        # Reconstruct the text with anonymized postcode, street, and city
        reconstructed_text = f"{fake_straatnaam} {fake_postcode} {fake_city} %1"
        return reconstructed_text
    else:
        return text




        # Find the index of the last occurrence of "%1"
        
        # if "%1" in parts:
        #     percent_index = parts.index("%1")
        #     # Find the index of the postcode
        #     postcode_index = None
        #     for i, part in enumerate(parts):
        #         if part.isnumeric():
        #             postcode_index = i
        #             break  
        #           # Extract the city name
        #     if postcode_index is not None and percent_index - 1 > postcode_index:
        #         city_code = parts[numeric_index][:2]
        #         city_index = parts.index(city_code)

        #         city = " ".join(parts[city_index: percent_index])  
        #         print(city) 
        #         if city not in  stad_to_fake_stad:
        #             fake_city = fake.city()
        #             stad_to_fake_stad[city] = fake_city    
        #         else:
        #             fake_city = stad_to_fake_stad[city] 
        # last_percent_index = len(parts) - 1 - parts[::-1].index("%1")
        # if last_percent_index:
        #     if last_percent_index - 2 >= 0:
        #         city = " ".join(parts[last_percent_index - 2:last_percent_index])
        #         if city not in stad_to_fake_stad:
        #             fake_city = fake.city()
        #             stad_to_fake_stad[city] = fake_city
        #         else: 
        #             fake_city = stad_to_fake_stad[city]
            

        
    #     new_parts = [
    #         fake_straatnaam,  # Anonymized street name
    #         fake_postcode,    # Anonymized postcode
    #         fake_city,
    #         "%1" # Rest of the text
    #     ]
        
    #     new_text = " ".join(new_parts)
    #     return new_text
    
    # else:
    #     return text








# _extract_postcode_street_anonymize("STIETSE VELDSTRAATWEG 109833 PA zwollo %1")
# print(_extract_postcode_street_anonymize("AGATHCHRISTIE SINGEL 1102 VW AMSTERDAM%1"))
print(_extract_postcode_street_anonymize("STIETSE VELDSTRAATWEG 2015 ZA  den hag%1"))
# print(_extract_postcode_street_anonymize("STIETSE VELDSTRAATWEG 2015 ZA  den hag %1"))
# print(postcode_to_fake_postcode)
# print(stad_to_fake_stad)
# print(straat_to_fake_straat)

Lisaweg 8002DK Raerd %1


In [1]:
import re

# Sample strings
import re

# Sample strings
strings = ["adfahf 1102 vw", "STIETSE VELDSTRAATWEG 109833 PA DEN HAG", "ewqqwer 2090 NH 51"]

# Regular expression pattern to match postcodes
postcode_pattern = r'\d{4}\s[a-zA-Z]{2}\b'

for string in strings:
    # Find all matches of the postcode pattern in the string
    matches = re.findall(postcode_pattern, string)
    
    if matches:
        print(f"Postcodes found in '{string}': {matches}")
    else:
        print(f"No postcodes found in '{string}'.")


Postcodes found in 'adfahf 1102 vw': ['1102 vw']
Postcodes found in 'STIETSE VELDSTRAATWEG 109833 PA DEN HAG': ['9833 PA']
Postcodes found in 'ewqqwer 2090 NH 51': ['2090 NH']


In [ ]:

print(data2.columns,data2.dtypes)

## data1
### quasi-identifiers
- rekening_number (account number) -> DEUTSCHE BANK NEDERLAND N.V. (example: NL20DEUT0319815528)
- rekening_naam (Account name) -> The Name of the client or account owner company (iven address of the company or location)
- tegenrekening_nummer (counteraccount number") -> This term is often used in banking and financial contexts to refer to the account number of a counterparty or a recipient when performing transactions, transfers, or payments.
- tegenrekening_naam (counteraccount name)-> it refers to the name associated with the counteraccount or recipient's bank account when performing transactions, transfers, or payments.
- omschrijving (description) -> info about banks (string of characters)
### Not important columns
- D/c -> empty
- munteenheid
- bedrag
- categorie
- bestandsnaam
## Data2
### quasi-identifiers
- rekening_nummer -> empty column
- rekening_naam -> empty column
- tegenrekening_naam -> empty column
- tegenrekening_nummer -> an attribute with string and integer values, (example: 9923635800,738-0156644-11, 000016200X95)
- 
### Not important columns
- Datum (date) 
- munteenheid
- bedrag
- D/C -> empty
- omschrijving -> Just a whole number
- categorie
- bestandsnaam
## Data 3
### quasi-identifiers
- id -> big whole number
- company -> name of the company
- Name -=> the name of the company (? what is the difference between company and name)
- straat -> 
- stad -> city where the client is located
- rekening -> just a whole number(? )

### Not important columns
- number -> house number, just a number
- toevoeging -> empty
- postcode -> just only number
- indienst -> empty
- uitdienst -> empty
- categorie
- bestandsnaam
## Data 4
### quasi-identifiers
- id
- company -> name of the company
- naam -> (?)
- straat
- stad
- rekening

### Not important columns
- nummer
-  toevoeging -> empty
- post code -> just a number
- indienst -> empty
- uitdienst -> empty
- categorie
- bestandsnaam
## Data 5
### quasi-identifiers
- id
- company
-  naam
- postcode
- stad
- rekening
- 
### Not important columns
-  nummer
- toevoeging
- indienst
- uitdienst
- categorie
- bestandsnaam



In [ ]:
df = pd.concat([data1, data2,data3,data4,data5],ignore_index = True)

In [ ]:
df

In [ ]:
size_in_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
print("Estimated DataFrame size:", size_in_mb, "MB")